In [651]:
import pandas as pd
import numpy as np
np.random.seed(1)

In [237]:
def mse_loss(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

In [669]:
class Neuron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias
        self.sum_neuron = 0
        
    def activation(self, x):
#         if name == sigmoid:
        return 1 / (1 + np.exp(-x))

    def deriv_sigmoid(self, x):
      # Производная сигмоиды: f'(x) = f(x) * (1 - f(x))
        fx = sigmoid(x)
        return fx * (1 - fx)
        
    def forward(self, input_vector):
        self.sum_neuron = (np.dot(self.weights, input_vector) + self.bias).ravel()
        return self.activation(self.sum_neuron).ravel()
    

In [670]:
class Linear:
#   '''Один слой нейронной сети'''
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.neurons = []
        
        for layer in range(output_size):
            np.random.seed(21) 
            self.weights = np.random.rand(1, input_size)
            np.random.seed(21) 
            self.bias = np.random.randn()
            self.neurons.append(Neuron(self.weights, self.bias))

    def forward(self, inputs):
        outputs = []
        for neuron in self.neurons:
            f = neuron.forward(inputs).ravel()
            outputs.append(f)
        return np.array(outputs).ravel()


In [671]:
def back_prop(y_true, y_pred, lr, layers, ls):
#     print(y_true, y_pred)    
    
    ypred = -2 * (y_true - y_pred[0])

    ypred = ypred
    h = []  # FIFO
    ls.append(1)
    for layer in layers:
        bias_temp = []
        for neuron in layer.neurons:
            deriv = neuron.deriv_sigmoid(neuron.sum_neuron)[0]
            neuron.bias = [(ypred * lr * deriv)]
            
            weights_temp = []
            count = 0 
            if layer != layers[0]:
                hh = h.pop()
            else: hh = 1
            for weight in neuron.weights[0]:
                
                h_temp = ls[layers.index(layer)+1][count]  #h1
                weight_temp = h_temp * deriv #w5
                
                h.append(weight_temp * deriv)  #h1_new
                weights_temp.append((weight - (ypred* lr * hh * weight_temp)))

                count+=1
            weights_temp = np.array(weights_temp).reshape(1, 2)
            neuron.weights = weights_temp               

In [672]:
linear1 = Linear(2,2)
linear2 = Linear(2,1)
learn_rate = 0.1
epochs = 1000

def mse_loss(y_true, y_pred):
    # y_true и y_pred - массивы numpy одинаковой длины.
    return ((y_true - y_pred) ** 2).mean()


def train(data, all_y_trues = None):
        
        for epoch in range(epochs):
            y_preds = []
            for x, y_true in zip(data, all_y_trues):
                l1 = linear1.forward(x)
                l2 = linear2.forward(l1)
                y_pred = l2
                back_prop(y_true, y_pred, learn_rate, [linear2, linear1], [l2, l1, x])
                y_preds.append(y_pred[0])

          # --- Считаем полные потери в конце каждой эпохи
            if epoch % 10 == 0:
                loss = mse_loss(all_y_trues, y_preds)
                print("Epoch %d loss: %.3f" % (epoch, loss))
                
def test(data):
#     for x in data:
    l1 = linear1.forward(data)
    l2 = linear2.forward(l1)
    y_pred = l2
    return y_pred[0]

In [673]:
# Определим набор данных
data = np.array([
  [-2, -1],  # Алиса
  [25, 6],   # Боб
  [17, 4],   # Чарли
  [-15, -6], # Диана
])
all_y_trues = np.array([
  1, # Алиса
  0, # Боб
  0, # Чарли
  1, # Диана
])

# Обучаем нашу нейронную сеть!
train(data, all_y_trues)

Epoch 0 loss: 0.283
Epoch 10 loss: 0.231
Epoch 20 loss: 0.174
Epoch 30 loss: 0.134
Epoch 40 loss: 0.107
Epoch 50 loss: 0.088
Epoch 60 loss: 0.074
Epoch 70 loss: 0.064
Epoch 80 loss: 0.055
Epoch 90 loss: 0.049
Epoch 100 loss: 0.044
Epoch 110 loss: 0.039
Epoch 120 loss: 0.036
Epoch 130 loss: 0.033
Epoch 140 loss: 0.030
Epoch 150 loss: 0.028
Epoch 160 loss: 0.026
Epoch 170 loss: 0.024
Epoch 180 loss: 0.022
Epoch 190 loss: 0.021
Epoch 200 loss: 0.020
Epoch 210 loss: 0.018
Epoch 220 loss: 0.017
Epoch 230 loss: 0.017
Epoch 240 loss: 0.016
Epoch 250 loss: 0.015
Epoch 260 loss: 0.014
Epoch 270 loss: 0.014
Epoch 280 loss: 0.013
Epoch 290 loss: 0.012
Epoch 300 loss: 0.012
Epoch 310 loss: 0.011
Epoch 320 loss: 0.011
Epoch 330 loss: 0.011
Epoch 340 loss: 0.010
Epoch 350 loss: 0.010
Epoch 360 loss: 0.010
Epoch 370 loss: 0.009
Epoch 380 loss: 0.009
Epoch 390 loss: 0.009
Epoch 400 loss: 0.008
Epoch 410 loss: 0.008
Epoch 420 loss: 0.008
Epoch 430 loss: 0.008
Epoch 440 loss: 0.007
Epoch 450 loss: 0.007

In [674]:
# Делаем пару предсказаний
emily = np.array([-7, -3]) # 128 фунтов (52.35 кг), 63 дюйма (160 см)
frank = np.array([20, 2])  # 155 pounds (63.4 кг), 68 inches (173 см)
print("Эмили: %.3f" % test(emily)) # 0.951 - Ж
print("Фрэнк: %.3f" % test(frank)) # 0.039 - М

Эмили: 0.980
Фрэнк: 0.045
